In [8]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [9]:
#  Load Base Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Build Model 
x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(5, activation='softmax')(x)  

model = models.Model(inputs=base_model.input, outputs=output)

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,813,381 (64.14 MB)

 Trainable params: 2,098,693 (8.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [15]:
#  Data Generators (Augmentation + Rescale) 
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"D:\depi_project\cats_dataset_balanced\train",
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    r"D:\depi_project\cats_dataset_balanced\val",
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'
)

Found 11513 images belonging to 5 classes.
Found 2880 images belonging to 5 classes.


In [16]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
checkpoint = ModelCheckpoint("best_vgg16_model.keras", monitor='val_loss', save_best_only=True, verbose=1)

In [17]:
# Train - Phase 1 (Feature Extraction)
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

C:\Users\metra\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 54/360 ━━━━━━━━━━━━━━━━━━━━ 9:12 2s/step - accuracy: 0.3054 - loss: 1.6832

C:\Users\metra\AppData\Roaming\Python\Python313\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4520 - loss: 1.3376
Epoch 1: val_loss improved from None to 0.91619, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 756s 2s/step - accuracy: 0.5252 - loss: 1.1786 - val_accuracy: 0.6306 - val_loss: 0.9162 - learning_rate: 1.0000e-04
Epoch 2/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6053 - loss: 0.9973
Epoch 2: val_loss improved from 0.91619 to 0.86013, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 866s 2s/step - accuracy: 0.6075 - loss: 0.9921 - val_accuracy: 0.6424 - val_loss: 0.8601 - learning_rate: 1.0000e-04
Epoch 3/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6227 - loss: 0.9501
Epoch 3: val_loss improved from 0.86013 to 0.81683, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 658s 2s/step - accuracy: 0.6280 - loss: 0.9398 - val_accuracy: 0.6639 - val_loss: 0.8168 - learning_rate: 1.0000e-04
Epoch 4/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/st

In [18]:
#  Unfreeze last 4 layers (Fine-tuning) 
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# Train - Phase 2 (Fine-tuning) 
history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

Epoch 1/20
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7048 - loss: 0.7497
Epoch 1: val_loss improved from 0.69658 to 0.69548, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 713s 2s/step - accuracy: 0.7104 - loss: 0.7201 - val_accuracy: 0.7163 - val_loss: 0.6955 - learning_rate: 1.0000e-05
Epoch 2/20
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7292 - loss: 0.6653
Epoch 2: val_loss improved from 0.69548 to 0.65915, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 740s 2s/step - accuracy: 0.7356 - loss: 0.6540 - val_accuracy: 0.7309 - val_loss: 0.6592 - learning_rate: 1.0000e-05
Epoch 3/20
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7466 - loss: 0.6190
Epoch 3: val_loss improved from 0.65915 to 0.64935, saving model to best_vgg16_model.keras
360/360 ━━━━━━━━━━━━━━━━━━━━ 750s 2s/step - accuracy: 0.7494 - loss: 0.6131 - val_accuracy: 0.7285 - val_loss: 0.6493 - learning_rate: 1.0000e-05
Epoch 4/20
360/360 ━━━━━━━━━━━━━━━

In [22]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

90/90 ━━━━━━━━━━━━━━━━━━━━ 254s 3s/step - accuracy: 0.8118 - loss: 0.4427
Validation Accuracy: 0.8118
Validation Loss: 0.4427


In [23]:
model.save("final_model.h5")

In [24]:
import os
print(os.getcwd())

d:\depi_project\code


In [25]:
import json

results = {
    "Validation Accuracy": 0.8118,
    "Validation Loss": 0.4427
}

with open("results.json", "w") as f:
    json.dump(results, f)
# الكود دا عشان يحفظ قيمة ال accuraccy , loss 
# وعشان المودل ميتعملوش رن مالاول تاني وياخد وقت كبير 
# اعمل رن بالكود دا عشان يفعله على جهازك
# from tensorflow.keras.models import load_model
# import json

# # Load model
# model = load_model("final_model.h5")

# # Load saved results
# with open("results.json", "r") as f:
#     results = json.load(f)

# print("Validation Accuracy:", results["Validation Accuracy"])
# print("Validation Loss:", results["Validation Loss"])

In [26]:
from tensorflow.keras.models import load_model
import json

# -------- Load trained model --------
model = load_model("final_model.h5")

# -------- Load saved results --------
with open("results.json", "r") as f:
    results = json.load(f)

print("✅ Model loaded successfully")
print("Validation Accuracy:", results["Validation Accuracy"])
print("Validation Loss:", results["Validation Loss"])

# -------- Example of using the model (optional) --------
# لو عايزين يعملوا predict على صورة أو داتا جديدة
# import numpy as np
# sample_input = np.random.rand(1, 128, 128, 3)  # مثال عشوائي
# prediction = model.predict(sample_input)
# print("Sample Prediction:", prediction)

✅ Model loaded successfully
Validation Accuracy: 0.8118
Validation Loss: 0.4427


In [30]:
import mlflow
import mlflow.keras
from tensorflow.keras.models import load_model

# Load saved model
model = load_model("best_vgg16_model.keras")

# Your saved results 
val_acc = 0.8118
val_loss = 0.4427

mlflow.set_experiment("Cats_Classification_VGG16")

with mlflow.start_run():
    mlflow.log_metric("val_accuracy", val_acc)
    mlflow.log_metric("val_loss", val_loss)
    mlflow.keras.log_model(model, "model")

print("✅ Model and results logged to MLflow successfully!")


2025/09/28 03:27:11 INFO mlflow.tracking.fluent: Experiment with name 'Cats_Classification_VGG16' does not exist. Creating a new experiment.
2025/09/28 03:27:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/28 03:27:12 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/09/28 03:27:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model and results logged to MLflow successfully!
